<a href="https://colab.research.google.com/github/piotrlnordea/bielik2/blob/main/bielik_moje_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
  <img src="https://huggingface.co/speakleash/Bielik-7B-Instruct-v0.1/raw/main/speakleash_cyfronet.png">
</p>

This notebook is designed to enable you to experiment with various types of model quantization. In this instance, we've implemented three exemplary methods: GPTQ, AWQ, and GGUF. Each of these methods offers a unique approach to reducing model size and computational requirements while striving to maintain, or even enhance, model performance. Through this interactive notebook, you're invited to explore the effects of these quantization techniques on different models, providing hands-on experience with the cutting-edge in model optimization technology. Whether you're aiming to deploy models on resource-constrained devices or simply looking to improve efficiency, this notebook serves as a practical guide to understanding and applying these advanced quantization methods.

In [ ]:
%%capture

!pip install accelerate -q
!pip install optimum -q
!pip install auto-gptq -q
!pip install ctransformers[cuda]

In [ ]:
import gc
import torch
import warnings

warnings.filterwarnings("ignore") # good for demo, not good idea for production :)

## **GPTQ**
Gradient Projection for Quantization (GPTQ)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "speakleash/Bielik-7B-Instruct-v0.1-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=False,
    revision="main"
)

pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')
gptq=True

tokenizer_config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
human_prompt = "Wymień trzy ciekawe miejscowości na lubelszczyznie i krótko je scharakteryzuj?"

human_prompt = """ na podstawie następującego tekstu napisz nad jaką rzeczką leży Nałeczów ?  --- Nałęczów (dawniej: Nałęczów-Zdrój) – miasto w województwie lubelskim, w powiecie puławskim, siedziba gminy miejsko-wiejskiej Nałęczów. Historycznie położony jest w Małopolsce (początkowo w ziemi sandomierskiej, a następnie w ziemi lubelskiej). Według danych GUS z 31 grudnia 2019 r. Nałęczów liczył 3753 mieszkańców[2].
Nałęczów to jedyne w Polsce uzdrowisko o profilu wyłącznie kardiologicznym[potrzebny przypis]. Leczy się tam przede wszystkim choroby: wieńcową, nadciśnienie tętnicze, nerwice serca i stany ogólnego wyczerpania psychofizycznego. W powrocie do zdrowia pomaga mikroklimat sprzyjający naturalnemu obniżeniu się ciśnienia tętniczego krwi oraz zmniejszeniu dolegliwości serca. Nałęczów posiada również dobre warunki dla rehabilitacji pacjentów po zawale serca i operacjach kardiochirurgicznych.
W centrum miasta znajduje się 25-hektarowy park Zdrojowy ze stawem i z historyczną zabudową: pałacem Małachowskich, sanatorium „Książę Józef”, Starymi Łazienkami, Werandkami, Domkiem Gotyckim. Do nowszych budynków stojących w tej okolicy należą: Dom Zdrojowy, „Pawilon Angielski”, sanatoria uzdrowiskowe: „Rolnik”, „Atrium”, „Termy Pałacowe”.
Położenie geograficzne
Nałęczów leży na Wyżynie Lubelskiej w środkowej części Płaskowyżu Nałęczowskiego. Jest częścią trójkąta turystycznego: Puławy – Kazimierz Dolny – Nałęczów[3]. Położony jest nad rzeką Bystrą, przy ujściu jej dopływu Bochotniczanki. Obszar miasta jest silnie zróżnicowany pod względem wysokości, pofałdowany i poprzecinany głębokimi wąwozami. Najwyżej położone są południowe i północne wzniesienia doliny, osiągające odpowiednio wysokości 212,2 (Las Zakładowy) oraz 207,3 m n.p.m. Centrum miasta z Parkiem Zdrojowym wznosi się na wysokości 170–175 m n.p.m.[4]
Zarys historii Nałęczowa
Początki sięgają przełomu VIII i IX wieku, gdy na obecnej Górze Poniatowskiego, wzgórzu górującym nad okolicą, wzniesiono gród. Później centrum osady przeniesiono na wzgórze, gdzie dziś znajduje się kościół parafialny. W I połowie XIV w. powstała parafia. Również w tym samym stuleciu dokonano lokacji wsi na prawie niemieckim. Pierwotnie miejscowość była nazywana Bochotnicą.
23 czerwca 1751 tereny te należące do Aleksandra Gałęzowskiego zakupił Stanisław Małachowski (starosta wąwolnicki), nazywając w 1772 roku od noszonego przez swój ród herbu Nałęcz całą posiadłość Nałęczowem. Po powstaniu styczniowym dobra nałęczowskie kupił Michał Górski.
Około 1800 odkryto lecznicze właściwości wód i złoża borowin, a po ich analizie chemicznej w 1817 roku potwierdził je prof. Piotr Celiński z Uniwersytetu Warszawskiego. Wyniki te wykorzystali trzej lekarze-sybiracy: Fortunat Nowicki, Wacław Lasocki i Konrad Chmielewski, modernizując w końcu XIX w. funkcjonujące już uzdrowisko za pieniądze ówczesnego właściciela Nałęczowa – Michała Górskiego. Stworzył on uzdrowisko w dzisiejszym kształcie miasta ogrodu. To on uczynił z Nałęczowa liczący się ośrodek leczniczo-kulturalny. Kuracjuszami byli tutaj m.in. Stefan Żeromski, Bolesław Prus i Henryk Sienkiewicz.
3 grudnia 1961 roku otwarto Muzeum Bolesława Prusa[5].
W 1963 roku Nałęczów otrzymał prawa miejskie. W latach 1975–1998 miasto należało administracyjnie do ówczesnego województwa lubelskiego.
"""
prompt = f"<s>[INST]{human_prompt}[/INST]"
print(prompt)

<s>[INST] na podstawie następującego tekstu napisz nad jaką rzeczką leży Nałeczów ?  --- Nałęczów (dawniej: Nałęczów-Zdrój) – miasto w województwie lubelskim, w powiecie puławskim, siedziba gminy miejsko-wiejskiej Nałęczów. Historycznie położony jest w Małopolsce (początkowo w ziemi sandomierskiej, a następnie w ziemi lubelskiej). Według danych GUS z 31 grudnia 2019 r. Nałęczów liczył 3753 mieszkańców[2].
Nałęczów to jedyne w Polsce uzdrowisko o profilu wyłącznie kardiologicznym[potrzebny przypis]. Leczy się tam przede wszystkim choroby: wieńcową, nadciśnienie tętnicze, nerwice serca i stany ogólnego wyczerpania psychofizycznego. W powrocie do zdrowia pomaga mikroklimat sprzyjający naturalnemu obniżeniu się ciśnienia tętniczego krwi oraz zmniejszeniu dolegliwości serca. Nałęczów posiada również dobre warunki dla rehabilitacji pacjentów po zawale serca i operacjach kardiochirurgicznych.
W centrum miasta znajduje się 25-hektarowy park Zdrojowy ze stawem i z historyczną zabudową: pałacem 

In [ ]:
outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    top_p=0.95
)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] na podstawie następującego tekstu napisz nad jaką rzeczką leży Nałeczów ?  --- Nałęczów (dawniej: Nałęczów-Zdrój) – miasto w województwie lubelskim, w powiecie puławskim, siedziba gminy miejsko-wiejskiej Nałęczów. Historycznie położony jest w Małopolsce (początkowo w ziemi sandomierskiej, a następnie w ziemi lubelskiej). Według danych GUS z 31 grudnia 2019 r. Nałęczów liczył 3753 mieszkańców[2].
Nałęczów to jedyne w Polsce uzdrowisko o profilu wyłącznie kardiologicznym[potrzebny przypis]. Leczy się tam przede wszystkim choroby: wieńcową, nadciśnienie tętnicze, nerwice serca i stany ogólnego wyczerpania psychofizycznego. W powrocie do zdrowia pomaga mikroklimat sprzyjający naturalnemu obniżeniu się ciśnienia tętniczego krwi oraz zmniejszeniu dolegliwości serca. Nałęczów posiada również dobre warunki dla rehabilitacji pacjentów po zawale serca i operacjach kardiochirurgicznych.
W centrum miasta znajduje się 25-hektarowy park Zdrojowy ze stawem i z historyczną zabudową: pałacem 

## **GGUF**
Group-wise Gradient Update Filtering (GGUF)

In [ ]:
# only if model, tokenizer is instantiate
if 'model' in globals() and 'tokenizer' in globals() and 'pipe' in globals():
  print("Remove from memory: model, tokenizer and pipe")
  del model, tokenizer, pipe
  gc.collect()
  torch.cuda.empty_cache()

Remove from memory: model, tokenizer and pipe


In [ ]:
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained(
    "speakleash/Bielik-7B-Instruct-v0.1-GGUF",
    model_file="bielik-7b-instruct-v0.1.Q4_K_M.gguf", # you can take different quantization resolution from speakleash/Bielik-7B-Instruct-v0.1-GGUF repo
    model_type="mistral",context_length=4096, gpu_layers=50, hf=True
)
tokenizer = AutoTokenizer.from_pretrained(
    "speakleash/Bielik-7B-Instruct-v0.1", use_fast=True
)

pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
human_prompt = "Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?"
prompt = f"<s>[INST]{human_prompt}[/INST]"
print(prompt)

<s>[INST]Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?[/INST]


In [ ]:
outputs = pipe(prompt, max_new_tokens=256)
print(outputs[0]["generated_text"])

<s>[INST]Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?[/INST]  Trzy ciekawe miejscowości w Polsce to:

1. Kraków - dawna stolica Polski, znana z zabytkowej architektury, w tym Wawelu, Sukiennic i Kościoła Mariackiego. Kraków jest również centrum kultury, oferującym festiwale, muzea i teatry.

2. Gdańsk - nadmorskie miasto z długą historią. Znane z Bazyliki Mariackiej, Dworu Artusa i fontanny Neptuna. Gdańsk jest również ważnym portem morskim i centrum biznesu.

3. Wrocław - miasto położone na Odrze, znane z licznych mostów i zabytkowej architektury. Wrocław jest Europejską Stolicą Kultury i słynie z festiwali, takich jak Wrocławski Festiwal Filmowy. Miasto jest również domem dla bogatej sieci park


In [ ]:
human_prompt = """ na podstawie następującego tekstu napisz nad jaką rzeczką leży Nałeczów ?  --- Nałęczów (dawniej: Nałęczów-Zdrój) – miasto w województwie lubelskim, w powiecie puławskim, siedziba gminy miejsko-wiejskiej Nałęczów. Historycznie położony jest w Małopolsce (początkowo w ziemi sandomierskiej, a następnie w ziemi lubelskiej). Według danych GUS z 31 grudnia 2019 r. Nałęczów liczył 3753 mieszkańców[2].
Nałęczów to jedyne w Polsce uzdrowisko o profilu wyłącznie kardiologicznym[potrzebny przypis]. Leczy się tam przede wszystkim choroby: wieńcową, nadciśnienie tętnicze, nerwice serca i stany ogólnego wyczerpania psychofizycznego. W powrocie do zdrowia pomaga mikroklimat sprzyjający naturalnemu obniżeniu się ciśnienia tętniczego krwi oraz zmniejszeniu dolegliwości serca. Nałęczów posiada również dobre warunki dla rehabilitacji pacjentów po zawale serca i operacjach kardiochirurgicznych.
W centrum miasta znajduje się 25-hektarowy park Zdrojowy ze stawem i z historyczną zabudową: pałacem Małachowskich, sanatorium „Książę Józef”, Starymi Łazienkami, Werandkami, Domkiem Gotyckim. Do nowszych budynków stojących w tej okolicy należą: Dom Zdrojowy, „Pawilon Angielski”, sanatoria uzdrowiskowe: „Rolnik”, „Atrium”, „Termy Pałacowe”.
Położenie geograficzne
Nałęczów leży na Wyżynie Lubelskiej w środkowej części Płaskowyżu Nałęczowskiego. Jest częścią trójkąta turystycznego: Puławy – Kazimierz Dolny – Nałęczów[3]. Położony jest nad rzeką Bystrą, przy ujściu jej dopływu Bochotniczanki. Obszar miasta jest silnie zróżnicowany pod względem wysokości, pofałdowany i poprzecinany głębokimi wąwozami. Najwyżej położone są południowe i północne wzniesienia doliny, osiągające odpowiednio wysokości 212,2 (Las Zakładowy) oraz 207,3 m n.p.m. Centrum miasta z Parkiem Zdrojowym wznosi się na wysokości 170–175 m n.p.m.[4]
Zarys historii Nałęczowa
Początki sięgają przełomu VIII i IX wieku, gdy na obecnej Górze Poniatowskiego, wzgórzu górującym nad okolicą, wzniesiono gród. Później centrum osady przeniesiono na wzgórze, gdzie dziś znajduje się kościół parafialny. W I połowie XIV w. powstała parafia. Również w tym samym stuleciu dokonano lokacji wsi na prawie niemieckim. Pierwotnie miejscowość była nazywana Bochotnicą.
23 czerwca 1751 tereny te należące do Aleksandra Gałęzowskiego zakupił Stanisław Małachowski (starosta wąwolnicki), nazywając w 1772 roku od noszonego przez swój ród herbu Nałęcz całą posiadłość Nałęczowem. Po powstaniu styczniowym dobra nałęczowskie kupił Michał Górski.
Około 1800 odkryto lecznicze właściwości wód i złoża borowin, a po ich analizie chemicznej w 1817 roku potwierdził je prof. Piotr Celiński z Uniwersytetu Warszawskiego. Wyniki te wykorzystali trzej lekarze-sybiracy: Fortunat Nowicki, Wacław Lasocki i Konrad Chmielewski, modernizując w końcu XIX w. funkcjonujące już uzdrowisko za pieniądze ówczesnego właściciela Nałęczowa – Michała Górskiego. Stworzył on uzdrowisko w dzisiejszym kształcie miasta ogrodu. To on uczynił z Nałęczowa liczący się ośrodek leczniczo-kulturalny. Kuracjuszami byli tutaj m.in. Stefan Żeromski, Bolesław Prus i Henryk Sienkiewicz.
3 grudnia 1961 roku otwarto Muzeum Bolesława Prusa[5].
W 1963 roku Nałęczów otrzymał prawa miejskie. W latach 1975–1998 miasto należało administracyjnie do ówczesnego województwa lubelskiego.
"""
prompt = f"[INST]{human_prompt}[/INST]"
print(prompt)
outputs = pipe(prompt, max_new_tokens=888)
print(outputs[0]["generated_text"])

[INST] na podstawie następującego tekstu napisz nad jaką rzeczką leży Nałeczów ?  --- Nałęczów (dawniej: Nałęczów-Zdrój) – miasto w województwie lubelskim, w powiecie puławskim, siedziba gminy miejsko-wiejskiej Nałęczów. Historycznie położony jest w Małopolsce (początkowo w ziemi sandomierskiej, a następnie w ziemi lubelskiej). Według danych GUS z 31 grudnia 2019 r. Nałęczów liczył 3753 mieszkańców[2].
Nałęczów to jedyne w Polsce uzdrowisko o profilu wyłącznie kardiologicznym[potrzebny przypis]. Leczy się tam przede wszystkim choroby: wieńcową, nadciśnienie tętnicze, nerwice serca i stany ogólnego wyczerpania psychofizycznego. W powrocie do zdrowia pomaga mikroklimat sprzyjający naturalnemu obniżeniu się ciśnienia tętniczego krwi oraz zmniejszeniu dolegliwości serca. Nałęczów posiada również dobre warunki dla rehabilitacji pacjentów po zawale serca i operacjach kardiochirurgicznych.
W centrum miasta znajduje się 25-hektarowy park Zdrojowy ze stawem i z historyczną zabudową: pałacem Mał

In [ ]:
human_prompt = """ na podstawie następującego tekstu napisz skąd się wziełaa nazwa Nałeczów ?  --- Nałęczów (dawniej: Nałęczów-Zdrój) – miasto w województwie lubelskim, w powiecie puławskim, siedziba gminy miejsko-wiejskiej Nałęczów. Historycznie położony jest w Małopolsce (początkowo w ziemi sandomierskiej, a następnie w ziemi lubelskiej). Według danych GUS z 31 grudnia 2019 r. Nałęczów liczył 3753 mieszkańców[2].
Nałęczów to jedyne w Polsce uzdrowisko o profilu wyłącznie kardiologicznym[potrzebny przypis]. Leczy się tam przede wszystkim choroby: wieńcową, nadciśnienie tętnicze, nerwice serca i stany ogólnego wyczerpania psychofizycznego. W powrocie do zdrowia pomaga mikroklimat sprzyjający naturalnemu obniżeniu się ciśnienia tętniczego krwi oraz zmniejszeniu dolegliwości serca. Nałęczów posiada również dobre warunki dla rehabilitacji pacjentów po zawale serca i operacjach kardiochirurgicznych.
W centrum miasta znajduje się 25-hektarowy park Zdrojowy ze stawem i z historyczną zabudową: pałacem Małachowskich, sanatorium „Książę Józef”, Starymi Łazienkami, Werandkami, Domkiem Gotyckim. Do nowszych budynków stojących w tej okolicy należą: Dom Zdrojowy, „Pawilon Angielski”, sanatoria uzdrowiskowe: „Rolnik”, „Atrium”, „Termy Pałacowe”.
Położenie geograficzne
Nałęczów leży na Wyżynie Lubelskiej w środkowej części Płaskowyżu Nałęczowskiego. Jest częścią trójkąta turystycznego: Puławy – Kazimierz Dolny – Nałęczów[3]. Położony jest nad rzeką Bystrą, przy ujściu jej dopływu Bochotniczanki. Obszar miasta jest silnie zróżnicowany pod względem wysokości, pofałdowany i poprzecinany głębokimi wąwozami. Najwyżej położone są południowe i północne wzniesienia doliny, osiągające odpowiednio wysokości 212,2 (Las Zakładowy) oraz 207,3 m n.p.m. Centrum miasta z Parkiem Zdrojowym wznosi się na wysokości 170–175 m n.p.m.[4]
Zarys historii Nałęczowa
Początki sięgają przełomu VIII i IX wieku, gdy na obecnej Górze Poniatowskiego, wzgórzu górującym nad okolicą, wzniesiono gród. Później centrum osady przeniesiono na wzgórze, gdzie dziś znajduje się kościół parafialny. W I połowie XIV w. powstała parafia. Również w tym samym stuleciu dokonano lokacji wsi na prawie niemieckim. Pierwotnie miejscowość była nazywana Bochotnicą.
23 czerwca 1751 tereny te należące do Aleksandra Gałęzowskiego zakupił Stanisław Małachowski (starosta wąwolnicki), nazywając w 1772 roku od noszonego przez swój ród herbu Nałęcz całą posiadłość Nałęczowem. Po powstaniu styczniowym dobra nałęczowskie kupił Michał Górski.
Około 1800 odkryto lecznicze właściwości wód i złoża borowin, a po ich analizie chemicznej w 1817 roku potwierdził je prof. Piotr Celiński z Uniwersytetu Warszawskiego. Wyniki te wykorzystali trzej lekarze-sybiracy: Fortunat Nowicki, Wacław Lasocki i Konrad Chmielewski, modernizując w końcu XIX w. funkcjonujące już uzdrowisko za pieniądze ówczesnego właściciela Nałęczowa – Michała Górskiego. Stworzył on uzdrowisko w dzisiejszym kształcie miasta ogrodu. To on uczynił z Nałęczowa liczący się ośrodek leczniczo-kulturalny. Kuracjuszami byli tutaj m.in. Stefan Żeromski, Bolesław Prus i Henryk Sienkiewicz.
3 grudnia 1961 roku otwarto Muzeum Bolesława Prusa[5].
W 1963 roku Nałęczów otrzymał prawa miejskie. W latach 1975–1998 miasto należało administracyjnie do ówczesnego województwa lubelskiego.
"""
prompt = f"[INST]{human_prompt}[/INST]"
print(prompt)
outputs = pipe(prompt, max_new_tokens=888)
print(outputs[0]["generated_text"])

[INST] na podstawie następującego tekstu napisz skąd się wziełaa nazwa Nałeczów ?  --- Nałęczów (dawniej: Nałęczów-Zdrój) – miasto w województwie lubelskim, w powiecie puławskim, siedziba gminy miejsko-wiejskiej Nałęczów. Historycznie położony jest w Małopolsce (początkowo w ziemi sandomierskiej, a następnie w ziemi lubelskiej). Według danych GUS z 31 grudnia 2019 r. Nałęczów liczył 3753 mieszkańców[2].
Nałęczów to jedyne w Polsce uzdrowisko o profilu wyłącznie kardiologicznym[potrzebny przypis]. Leczy się tam przede wszystkim choroby: wieńcową, nadciśnienie tętnicze, nerwice serca i stany ogólnego wyczerpania psychofizycznego. W powrocie do zdrowia pomaga mikroklimat sprzyjający naturalnemu obniżeniu się ciśnienia tętniczego krwi oraz zmniejszeniu dolegliwości serca. Nałęczów posiada również dobre warunki dla rehabilitacji pacjentów po zawale serca i operacjach kardiochirurgicznych.
W centrum miasta znajduje się 25-hektarowy park Zdrojowy ze stawem i z historyczną zabudową: pałacem Ma

In [ ]:
human_prompt = """ na podstawie następującego tekstu napisz kim byli lekarze sybiracy ?  --- Nałęczów (dawniej: Nałęczów-Zdrój) – miasto w województwie lubelskim, w powiecie puławskim, siedziba gminy miejsko-wiejskiej Nałęczów. Historycznie położony jest w Małopolsce (początkowo w ziemi sandomierskiej, a następnie w ziemi lubelskiej). Według danych GUS z 31 grudnia 2019 r. Nałęczów liczył 3753 mieszkańców[2].
Nałęczów to jedyne w Polsce uzdrowisko o profilu wyłącznie kardiologicznym[potrzebny przypis]. Leczy się tam przede wszystkim choroby: wieńcową, nadciśnienie tętnicze, nerwice serca i stany ogólnego wyczerpania psychofizycznego. W powrocie do zdrowia pomaga mikroklimat sprzyjający naturalnemu obniżeniu się ciśnienia tętniczego krwi oraz zmniejszeniu dolegliwości serca. Nałęczów posiada również dobre warunki dla rehabilitacji pacjentów po zawale serca i operacjach kardiochirurgicznych.
W centrum miasta znajduje się 25-hektarowy park Zdrojowy ze stawem i z historyczną zabudową: pałacem Małachowskich, sanatorium „Książę Józef”, Starymi Łazienkami, Werandkami, Domkiem Gotyckim. Do nowszych budynków stojących w tej okolicy należą: Dom Zdrojowy, „Pawilon Angielski”, sanatoria uzdrowiskowe: „Rolnik”, „Atrium”, „Termy Pałacowe”.
Położenie geograficzne
Nałęczów leży na Wyżynie Lubelskiej w środkowej części Płaskowyżu Nałęczowskiego. Jest częścią trójkąta turystycznego: Puławy – Kazimierz Dolny – Nałęczów[3]. Położony jest nad rzeką Bystrą, przy ujściu jej dopływu Bochotniczanki. Obszar miasta jest silnie zróżnicowany pod względem wysokości, pofałdowany i poprzecinany głębokimi wąwozami. Najwyżej położone są południowe i północne wzniesienia doliny, osiągające odpowiednio wysokości 212,2 (Las Zakładowy) oraz 207,3 m n.p.m. Centrum miasta z Parkiem Zdrojowym wznosi się na wysokości 170–175 m n.p.m.[4]
Zarys historii Nałęczowa
Początki sięgają przełomu VIII i IX wieku, gdy na obecnej Górze Poniatowskiego, wzgórzu górującym nad okolicą, wzniesiono gród. Później centrum osady przeniesiono na wzgórze, gdzie dziś znajduje się kościół parafialny. W I połowie XIV w. powstała parafia. Również w tym samym stuleciu dokonano lokacji wsi na prawie niemieckim. Pierwotnie miejscowość była nazywana Bochotnicą.
23 czerwca 1751 tereny te należące do Aleksandra Gałęzowskiego zakupił Stanisław Małachowski (starosta wąwolnicki), nazywając w 1772 roku od noszonego przez swój ród herbu Nałęcz całą posiadłość Nałęczowem. Po powstaniu styczniowym dobra nałęczowskie kupił Michał Górski.
Około 1800 odkryto lecznicze właściwości wód i złoża borowin, a po ich analizie chemicznej w 1817 roku potwierdził je prof. Piotr Celiński z Uniwersytetu Warszawskiego. Wyniki te wykorzystali trzej lekarze-sybiracy: Fortunat Nowicki, Wacław Lasocki i Konrad Chmielewski, modernizując w końcu XIX w. funkcjonujące już uzdrowisko za pieniądze ówczesnego właściciela Nałęczowa – Michała Górskiego. Stworzył on uzdrowisko w dzisiejszym kształcie miasta ogrodu. To on uczynił z Nałęczowa liczący się ośrodek leczniczo-kulturalny. Kuracjuszami byli tutaj m.in. Stefan Żeromski, Bolesław Prus i Henryk Sienkiewicz.
3 grudnia 1961 roku otwarto Muzeum Bolesława Prusa[5].
W 1963 roku Nałęczów otrzymał prawa miejskie. W latach 1975–1998 miasto należało administracyjnie do ówczesnego województwa lubelskiego.
"""
prompt = f"[INST]{human_prompt}[/INST]"
print(prompt)
outputs = pipe(prompt, max_new_tokens=888)
print(outputs[0]["generated_text"])

[INST] na podstawie następującego tekstu napisz kim byli lekarze sybiracy ?  --- Nałęczów (dawniej: Nałęczów-Zdrój) – miasto w województwie lubelskim, w powiecie puławskim, siedziba gminy miejsko-wiejskiej Nałęczów. Historycznie położony jest w Małopolsce (początkowo w ziemi sandomierskiej, a następnie w ziemi lubelskiej). Według danych GUS z 31 grudnia 2019 r. Nałęczów liczył 3753 mieszkańców[2].
Nałęczów to jedyne w Polsce uzdrowisko o profilu wyłącznie kardiologicznym[potrzebny przypis]. Leczy się tam przede wszystkim choroby: wieńcową, nadciśnienie tętnicze, nerwice serca i stany ogólnego wyczerpania psychofizycznego. W powrocie do zdrowia pomaga mikroklimat sprzyjający naturalnemu obniżeniu się ciśnienia tętniczego krwi oraz zmniejszeniu dolegliwości serca. Nałęczów posiada również dobre warunki dla rehabilitacji pacjentów po zawale serca i operacjach kardiochirurgicznych.
W centrum miasta znajduje się 25-hektarowy park Zdrojowy ze stawem i z historyczną zabudową: pałacem Małachow

## **AWQ**
Adaptive Weight Quantization (AWQ)

**NOTE**: To use this example of `AWQ`, we first need to disconnect the runtime before installing the `vllm` dependency. There can be dependency conflicts with the packages we installed previously, so it is best to start from scratch.

In [ ]:
# restart Colab env
exit()

In [ ]:
%%capture

!pip install vllm

In [ ]:
from vllm import LLM, SamplingParams

sampling_params = SamplingParams(temperature=0.0, top_p=1.0, max_tokens=256)
llm = LLM(
    model="speakleash/Bielik-7B-Instruct-v0.1-AWQ",
    quantization='awq',
    dtype='half',
    gpu_memory_utilization=.95,
    max_model_len=4096
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

WARNING 04-04 10:18:40 config.py:211] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 04-04 10:18:40 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='speakleash/Bielik-7B-Instruct-v0.1-AWQ', tokenizer='speakleash/Bielik-7B-Instruct-v0.1-AWQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


tokenizer_config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

INFO 04-04 10:18:47 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 04-04 10:18:47 selector.py:25] Using XFormers backend.
INFO 04-04 10:18:51 weight_utils.py:177] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

INFO 04-04 10:19:39 model_runner.py:104] Loading model weights took 3.8745 GB
INFO 04-04 10:19:43 gpu_executor.py:94] # GPU blocks: 1035, # CPU blocks: 2048
INFO 04-04 10:19:47 model_runner.py:791] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-04 10:19:47 model_runner.py:795] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 04-04 10:20:02 model_runner.py:867] Graph capturing finished in 15 secs.


In [ ]:
human_prompt = "Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?"
prompt = f"<s>[INST]{human_prompt}[/INST]"
print(prompt)

<s>[INST]Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?[/INST]


In [ ]:
output = llm.generate(prompt, sampling_params)
print(output[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.84s/it]

  Trzy ciekawe miejscowości w Polsce to:
1. Gdańsk - nadmorskie miasto z bogatą historią i kulturą, znane z Bazyliki Mariackiej, Dworu Artusa, fontanny Neptuna i Muzeum II Wojny Światowej.
2. Kraków - historyczne miasto z piękną starówką, Wawelskim Zamkiem, Sukiennicami i Kościołem Mariackim.
3. Wrocław - miasto z malowniczymi kanałami, Starym Rynkiem i zabytkami takimi jak Ostrów Tumski, Most Pokoju i Halą Stulecia.
